<a href="https://colab.research.google.com/github/jcandane/pykry/blob/main/fcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def strr(X):
  return np.format_float_positional(X, unique=False, precision=5, pad_left=True)

$\left( \sum_x \left( \frac{R^2_{ix}}{\left( a \ell_x/2\right)^2} \right)_{ix} \right)_i < 1$

In [2]:
def FCClattice(a, la, lb, lc, sphere=False, element=None):
    """
    GIVEN:  a lattice constant
            la, lb, lc (number of unit cells in each direction)
            **sphere (option to get ellipsoid cutout)
            **element (option to give element type)
    GET:    R_ix, all positions in the full crystal
            **Z, return array with element type
    """

    R_ix_in  = np.array([[0.,0.,0.],[np.sqrt(2)/2,np.sqrt(2)/2,0.],[np.sqrt(2)/2.,0.,np.sqrt(2)/2.],[0., np.sqrt(2)/2.,np.sqrt(2)/2.]])
    R_ix_in *= a/np.sqrt(2)

    x_   = a * np.arange(0, la, 1)
    y_   = a * np.arange(0, lb, 1)
    z_   = a * np.arange(0, lc, 1)
    R_Ix = np.array(np.meshgrid(x_, y_, z_))
    R_Ix = R_Ix.reshape( (3, len(x_)*len(y_)*len(z_)) , order="C").T

    if sphere:
        center = np.array([la*a, lb*a, lc*a])/2
        ll = (np.array([la, lb, lc]) * a/2)**2
        R_Ix = R_Ix[ np.sum( ( R_Ix - center[None,:] )**2 / ll[None,:], axis = 1 ) < 1 ]

    R_ix = (R_ix_in[None, :, :] + R_Ix[:,None,:]).reshape((R_Ix.shape[0]*R_ix_in.shape[0], 3), order="C")

    if element is not None:
        return np.tile(element, len(R_ix)), R_ix
    else:
        return R_ix

In [3]:
### inputs
å = 5.26 ## Å, lattice parameter
l = 33

la = l
lb = l
lc = l

center = np.array([la*å, lb*å, lc*å])/2

x_  = np.arange(0, la, 1)*å
y_  = np.arange(0, lb, 1)*å
z_  = np.arange(0, lc, 1)*å
xyz = np.array(np.meshgrid(x_, y_, z_))

R_Ix = xyz.reshape( (3, len(x_)*len(y_)*len(z_)) , order="C").T

Chop to get Sphere

In [4]:
R_Ix = R_Ix[ np.linalg.norm( R_Ix - center[None,:], axis=1) < l*å/2 ]
R_Ix.shape

(18656, 3)

Get Unit Cell Vectors

In [5]:
R_ix_in = np.array([[0.,0.,0.],[np.sqrt(2)/2,np.sqrt(2)/2,0.],[np.sqrt(2)/2.,0.,np.sqrt(2)/2.],[0., np.sqrt(2)/2.,np.sqrt(2)/2.]])* (å/np.sqrt(2))
R_ix_in

array([[0.  , 0.  , 0.  ],
       [2.63, 2.63, 0.  ],
       [2.63, 0.  , 2.63],
       [0.  , 2.63, 2.63]])

In [6]:
R_Iix = (R_ix_in[None, :, :] + R_Ix[:,None,:])

R_ix = R_Iix.reshape((R_Ix.shape[0]*R_ix_in.shape[0], 3), order="C")
R_ix.shape

(74624, 3)

In [7]:
string = str(len(R_ix)) + "\n\t\n"
for i in range(len(R_ix)):
    string += "Ar  " + strr(R_ix[i,0]) + "  " + strr(R_ix[i,1]) + "  " + strr(R_ix[i,2]) + " \n"

file1 = open("argoncrystal.xyz", "w")  # write mode
file1.write(string)
file1.close()

In [8]:
Z, R_ix = FCClattice(5.26, 60, 30, 30, sphere=True, element=18)

string = str(len(R_ix)) + "\n\t\n"
for i in range(len(R_ix)):
    #string += "Ar  " + strr(R_ix[i,0]) + "  " + strr(R_ix[i,1]) + "  " + strr(R_ix[i,2]) + " \n"
    string += str(Z[i]) + "  " + strr(R_ix[i,0]) + "  " + strr(R_ix[i,1]) + "  " + strr(R_ix[i,2]) + " \n"

file1 = open("argoncrystal.xyz", "w")  # write mode
file1.write(string)
file1.close()

In [9]:
!git clone https://github.com/jcandane/pyprototyp
from pyprototyp.pyprototyp import pyprototyp
pyprototyp("fcc", packages=[], useful=[], defs=[FCClattice], classes=[])

Cloning into 'pyprototyp'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 32 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.
